In [22]:
import pandas as pd

### load table countrycode - countrycoords and preprocess

In [23]:
df_co_from_cc = pd.read_csv('./data/location_ref/councode_councood.csv')

# reame columns
df_co_from_cc = df_co_from_cc.rename({'alpha2Code': 'COUNTRY_ALPHA2_CODE'}, axis='columns')
df_co_from_cc.head()

,country,COUNTRY_ALPHA2_CODE,countryCoords
0,http://www.wikidata.org/entity/Q5785,KY,Point(-80.5 19.5)
1,http://www.wikidata.org/entity/Q963,BW,Point(23.7 -22.2)
2,http://www.wikidata.org/entity/Q924,TZ,Point(34.854 -6.307)
3,http://www.wikidata.org/entity/Q117,GH,Point(-0.5 8.0)
4,http://www.wikidata.org/entity/Q16644,MP,Point(145.78 16.705)


In [24]:
df_co_from_cc['countryCoords'] = df_co_from_cc['countryCoords'].replace("Point\(", "", regex=True).replace(" ", ", ", regex=True).replace("\)", "", regex=True)


# converting to string series 
df_co_from_cc['countryCoords']= df_co_from_cc['countryCoords'].astype(str) 

# splitting at occurrence of whitespace 
df_co_from_cc['countryCoords']= df_co_from_cc['countryCoords'].str.split(",", 1) 
  
# displaying second element from list 
df_co_from_cc['country_lat_3857']  = df_co_from_cc['countryCoords'].str.get(1) 

# displaying first element from list 
df_co_from_cc['country_long_3857']  = df_co_from_cc['countryCoords'].str.get(0)
 
    
df_co_from_cc['country_lat_long_3857'] = df_co_from_cc['country_lat_3857'] + "," + df_co_from_cc['country_long_3857'] 

df_co_from_cc['country_lat_long_3857'] = df_co_from_cc['country_lat_long_3857'].replace(" ", "", regex=True)


df_co_from_cc = df_co_from_cc.drop(columns=['countryCoords', 'country'])

In [25]:
df_co_from_cc.head()

,COUNTRY_ALPHA2_CODE,country_lat_3857,country_long_3857,country_lat_long_3857
0,KY,19.5,-80.5,"19.5,-80.5"
1,BW,-22.2,23.7,"-22.2,23.7"
2,TZ,-6.307,34.854,"-6.307,34.854"
3,GH,8.0,-0.5,"8.0,-0.5"
4,MP,16.705,145.78,"16.705,145.78"


In [26]:
df_co_from_cc.shape

(256, 4)

### load table country names multilingual - country name - country code

In [27]:
df_c_multilingu = pd.read_csv('./data/location_ref/IP2LOCATION-COUNTRY-MULTILINGUAL.CSV')
df_c_multilingu = df_c_multilingu.drop(columns=['COUNTRY_ALPHA3_CODE', 'COUNTRY_NUMERIC_CODE'])
# make COUNTRY_NAME lowercase
df_c_multilingu['COUNTRY_NAME'] = df_c_multilingu['COUNTRY_NAME'].str.lower()
df_c_multilingu['LANG'] = df_c_multilingu['LANG'].str.lower()

df_c_multilingu.head()

,LANG,LANG_NAME,COUNTRY_ALPHA2_CODE,COUNTRY_NAME
0,af,AFRIKAANS,AD,andorra
1,af,AFRIKAANS,AE,verenigde arabiese emirate
2,af,AFRIKAANS,AF,afganistan
3,af,AFRIKAANS,AG,antigua en barbuda
4,af,AFRIKAANS,AI,anguilla


In [28]:
#drop: re, man
df_c_multilingu[df_c_multilingu['COUNTRY_NAME']=='man']

,LANG,LANG_NAME,COUNTRY_ALPHA2_CODE,COUNTRY_NAME
13051,nb,NORWEGIAN BOKMAL,IM,man
13798,nn,NORWEGIAN NYNORSK,IM,man
14047,no,NORWEGIAN,IM,man


In [29]:
df_c_multilingu[df_c_multilingu['COUNTRY_NAME']=='mf'] 
# yt, re, man, mf
# nga = russia

,LANG,LANG_NAME,COUNTRY_ALPHA2_CODE,COUNTRY_NAME
19562,vi,VIETNAMESE,MF,mf


In [30]:
df_c_multilingu = df_c_multilingu.drop(index=[13051,13798, 14047, 19609,19667,19562])

In [31]:
df_c_multilingu.shape #20169 #20166 # 20165

(20163, 4)

In [32]:
df_c_multilingu['COUNTRY_NAME'].nunique()

13387

In [33]:
# remove duplicates
df_c_multilingu = df_c_multilingu[~df_c_multilingu['COUNTRY_NAME'].duplicated()]

In [34]:
df_c_multilingu.shape

(13387, 4)

### Merge dataframes

In [35]:
# merge these 2 
df_c = df_c_multilingu.merge(df_co_from_cc, how='left', on='COUNTRY_ALPHA2_CODE',suffixes=(False, False))
df_c.shape

(13745, 7)

In [36]:
df_c.head()

,LANG,LANG_NAME,COUNTRY_ALPHA2_CODE,COUNTRY_NAME,country_lat_3857,country_long_3857,country_lat_long_3857
0,af,AFRIKAANS,AD,andorra,42.558333333,1.555277777,"42.558333333,1.555277777"
1,af,AFRIKAANS,AE,verenigde arabiese emirate,24.4,54.3,"24.4,54.3"
2,af,AFRIKAANS,AF,afganistan,34.0,66.0,"34.0,66.0"
3,af,AFRIKAANS,AG,antigua en barbuda,17.116666666,-61.85,"17.116666666,-61.85"
4,af,AFRIKAANS,AI,anguilla,18.227222222,-63.048888888,"18.227222222,-63.048888888"


In [37]:
df_c.shape

(13745, 7)

In [38]:
# check for dupliates in COUNTRY_NAME col
# remove duplicates cityLabel entries
df_c = df_c[~df_c['COUNTRY_NAME'].duplicated()]

In [39]:
df_c.shape

(13387, 7)

In [40]:
df_c.to_csv('./data/location_ref/pre_country_multilingu_countrycode_countrycoord.csv')

### import cityname multilingu

In [90]:
df_d = pd.read_csv('./data/location_ref/citynamemulti_councode_cityco_citypop.csv', 
                   dtype={'cityCoords': str },index_col=0)

In [91]:
df_d.head()

,language,CITY_NAME,COUNTRY_ALPHA2_CODE,population,city_lat_3857,city_long_3857,city_lat_long_3857
81,pl,Nowy Jork,US,8398748.0,40.67,-73.94,"40.67, -73.94"
82,en,New York City,US,8398748.0,40.67,-73.94,"40.67, -73.94"
83,fi,New York,US,8398748.0,40.67,-73.94,"40.67, -73.94"
84,he,ניו יורק,US,8398748.0,40.67,-73.94,"40.67, -73.94"
88,ru,Нью-Йорк,US,8398748.0,40.67,-73.94,"40.67, -73.94"


In [92]:
df_e = df_d.merge(df_co_from_cc, how='left', on='COUNTRY_ALPHA2_CODE',suffixes=(False, False))


In [93]:
# cols = list(df_e.columns.values)
col_order = ['language',
 'CITY_NAME',
 'city_lat_3857',
 'city_long_3857',
 'city_lat_long_3857',
 'population',
 'COUNTRY_ALPHA2_CODE',
 'country_lat_3857',
 'country_long_3857',
 'country_lat_long_3857']

In [94]:
df_e = df_e[col_order]

In [95]:
# make city_name lowercase
df_e['CITY_NAME'] = df_e['CITY_NAME'].str.lower()


In [96]:
df_e[2000:2100]

,language,CITY_NAME,city_lat_3857,city_long_3857,city_lat_long_3857,population,COUNTRY_ALPHA2_CODE,country_lat_3857,country_long_3857,country_lat_long_3857
2000,ba,нанкин,32.050000,118.766667,"32.05, 118.766666666",6320000.0,CN,35.844722222,103.451944444,"35.844722222,103.451944444"
2001,bg,нандзин,32.050000,118.766667,"32.05, 118.766666666",6320000.0,CN,35.844722222,103.451944444,"35.844722222,103.451944444"
2002,bn,নানচিং,32.050000,118.766667,"32.05, 118.766666666",6320000.0,CN,35.844722222,103.451944444,"35.844722222,103.451944444"
2003,bxr,нанжин,32.050000,118.766667,"32.05, 118.766666666",6320000.0,CN,35.844722222,103.451944444,"35.844722222,103.451944444"
2004,ca,nanquín,32.050000,118.766667,"32.05, 118.766666666",6320000.0,CN,35.844722222,103.451944444,"35.844722222,103.451944444"
...,...,...,...,...,...,...,...,...,...,...
2095,new,वार्सः,52.216667,21.033333,"52.216666666, 21.033333333",1790658.0,PL,52.0,19.0,"52.0,19.0"
2096,nn,warszawa,52.216667,21.033333,"52.216666666, 21.033333333",1790658.0,PL,52.0,19.0,"52.0,19.0"
2097,nov,varsava,52.216667,21.033333,"52.216666666, 21.033333333",1790658.0,PL,52.0,19.0,"52.0,19.0"
2098,nrm,varsovie,52.216667,21.033333,"52.216666666, 21.033333333",1790658.0,PL,52.0,19.0,"52.0,19.0"


In [97]:
df_e.to_csv('./data/location_ref/city_multilingu_citycoord_countrycode_countrycoord.csv')